In [8]:
import gradio as gr
import torch
from torchvision.transforms import ToTensor
from PIL import Image
from StereoNet import StereoNet

In [9]:
# Load your PyTorch model

if torch.cuda.is_available():
    # model = torch.load("./best-model.ckpt")
    model = StereoNet.load_from_checkpoint("./best-model.ckpt")
    # model = torch.load("./src/ui/best-model.ckpt")
else:
    model = StereoNet.load_from_checkpoint("./best-model.ckpt", map_location=torch.device('cpu'))
    # model = torch.load("./best-model.ckpt", map_location=torch.device('cpu'))
    # model = torch.load("./src/ui/best-model.ckpt", map_location=torch.device('cpu'))
# model = torch.load("../model/BEST-epoch=23-step=864006.ckpt")
# model.eval()

Lightning automatically upgraded your loaded checkpoint from v1.7.2 to v2.0.7. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file d:\__repos\stereo_net\src\ui\best-model.ckpt`


In [10]:
class Rescale():
    """
    Rescales the left and right image tensors (initially ranged between [0, 1]) and rescales them to be between [-1, 1].
    """

    @staticmethod
    def __call__(sample):
        for name in ['left', 'right']:
            sample[name] = (sample[name] - 0.5) * 2
        return sample
    
def png_loader(path):
    img = Image.open(path)
    return img

In [29]:
# Function to convert PNG to WebP
def prepare_sample(left_image, right_imate):

    # left_image = png_loader(left_image)
    # right_imate = png_loader(right_imate)
    # transforms = [Rescale(), ToTensor()]
    left_image = ToTensor()(left_image)
    right_imate = ToTensor()(right_imate)
    left_image = left_image.unsqueeze(0)
    right_imate = right_imate.unsqueeze(0)
    transforms = [Rescale()]

    sample = {
        'left': left_image, 'right': right_imate, 
        }

    # torch_sample = ToTensor()(sample)
    for transform in transforms:
        sample = transform(sample)
        # left_image = transform(left_image)
        # right_imate = transform(right_imate)


    return sample

def disparity_estimation(left_image, right_image):

    sample = prepare_sample(left_image, right_image)
    # disparity_map = model(sample['left'], sample['right'])
    with torch.no_grad():
        disparity_map = model(sample)
    
    print(type(disparity_map))
    print(disparity_map.shape)
    print(disparity_map.max)
    print(disparity_map.min)

    disparity_map = disparity_map.squeeze().numpy()

    return disparity_map

In [30]:
# Create a Gradio interface
iface = gr.Interface(
    fn=disparity_estimation,
    inputs=[
        gr.inputs.Image(type="pil", label="Left Image (PNG)"),
        gr.inputs.Image(type="pil", label="Right Image (PNG)"),
    ],
    outputs=gr.outputs.Image(type="pil", label="Disparity Map (PNG)"),
    title="Disparity Estimation",
    description="Upload two PNG images for left and right views to estimate disparity.",
)

C:\Users\Maciek\AppData\Local\Temp\ipykernel_15240\874370193.py:5: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Image(type="pil", label="Left Image (PNG)"),
C:\Users\Maciek\AppData\Local\Temp\ipykernel_15240\874370193.py:5: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Image(type="pil", label="Left Image (PNG)"),
C:\Users\Maciek\AppData\Local\Temp\ipykernel_15240\874370193.py:6: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  gr.inputs.Image(type="pil", label="Right Image (PNG)"),
C:\Users\Maciek\AppData\Local\Temp\ipykernel_15240\874370193.py:6: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  gr.inputs.Image(type="pil", label="Right Image (PNG)"),
C:\Users\Maciek\AppDat

In [31]:
# Start the Gradio UI
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


<class 'torch.Tensor'>
torch.Size([1, 1, 540, 960])


Traceback (most recent call last):
  File "d:\__repos\stereo_net\venv\lib\site-packages\gradio\routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
  File "d:\__repos\stereo_net\venv\lib\site-packages\gradio\blocks.py", line 1434, in process_api
    data = self.postprocess_data(fn_index, result["prediction"], state)
  File "d:\__repos\stereo_net\venv\lib\site-packages\gradio\blocks.py", line 1341, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
  File "d:\__repos\stereo_net\venv\lib\site-packages\gradio\components\image.py", line 314, in postprocess
    return processing_utils.encode_array_to_base64(y)
  File "d:\__repos\stereo_net\venv\lib\site-packages\gradio\processing_utils.py", line 104, in encode_array_to_base64
    pil_image = Image.fromarray(_convert(image_array, np.uint8, force_copy=False))
  File "d:\__repos\stereo_net\venv\lib\site-packages\gradio\processing_utils.py", line 416, in _convert
    raise ValueError(

In [28]:
iface.close()

Closing server running on port: 7860
